In [1]:
# First prototype for an SEC filings scraper

In [ ]:
# Notes:
    # On the SEC, max requests rate is 10 per second
    # They ask to give a user agent with contact details

# URL's:
    # https://www.sec.gov/files/company_tickers_exchange.json
    # https://data.sec.gov/submissions/CIK{cik}.json
    # https://data.sec.gov/submissions/{submission_id}"
    # https://www.sec.gov/Archives/edgar/data/{cik}/{submission_id}/
    # https://www.sec.gov/Archives/edgar/data/{cik}


In [ ]:
# TODO: Take a look at possible SEC data sources
    # https://www.sec.gov/search-filings/edgar-search-assistance/accessing-edgar-data
    # https://www.sec.gov/search-filings/edgar-application-programming-interfaces
    # https://api-bravo.edgarfiling.sec.gov/